In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,26132
2,Huelva,Confirmados PDIA 14 días,987
3,Huelva,Tasa PDIA 14 días,"192,33392443050062"
4,Huelva,Confirmados PDIA 7 días,574
5,Huelva,Total Confirmados,26330
6,Huelva,Curados,21546
7,Huelva,Fallecidos,355


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  26132.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6827.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1209 personas en los últimos 7 días 

Un positivo PCR cada 827 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,26132.0,574.0,987.0,513170.0,111.853772,192.333924,147.0
Huelva-Costa,15624.0,223.0,341.0,289548.0,77.016591,117.769765,70.0
Condado-Campiña,7726.0,308.0,584.0,156231.0,197.143973,373.805455,64.0
Huelva (capital),6827.0,119.0,174.0,143837.0,82.732538,120.970265,33.0
Cartaya,1063.0,56.0,78.0,20083.0,278.842802,388.388189,26.0
Almonte,1106.0,89.0,174.0,24507.0,363.161546,710.001224,22.0
Moguer,929.0,42.0,69.0,21867.0,192.070243,315.543970,10.0
Sierra de Huelva-Andévalo Central,2353.0,29.0,42.0,67391.0,43.032452,62.322862,10.0
San Juan del Puerto,393.0,19.0,26.0,9411.0,201.891404,276.272447,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Bartolomé de la Torre,252.0,21.0,38.0,3761.0,558.362138,1010.369583,3.0
Rociana del Condado,391.0,21.0,66.0,7939.0,264.516942,831.338960,2.0
Almonte,1106.0,89.0,174.0,24507.0,363.161546,710.001224,22.0
Bollullos Par del Condado,695.0,29.0,58.0,14387.0,201.570863,403.141725,2.0
Niebla,178.0,5.0,16.0,4116.0,121.477162,388.726919,1.0
Cartaya,1063.0,56.0,78.0,20083.0,278.842802,388.388189,26.0
Bonares,203.0,17.0,23.0,6060.0,280.528053,379.537954,4.0
Condado-Campiña,7726.0,308.0,584.0,156231.0,197.143973,373.805455,64.0
Palos de la Frontera,503.0,23.0,39.0,11742.0,195.878045,332.141032,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Isla Cristina,2013.0,2.0,7.0,21393.0,9.348852,32.720983,0.0,0.285714
Gibraleón,552.0,6.0,20.0,12737.0,47.106854,157.022847,0.0,0.300000
Niebla,178.0,5.0,16.0,4116.0,121.477162,388.726919,1.0,0.312500
Rociana del Condado,391.0,21.0,66.0,7939.0,264.516942,831.338960,2.0,0.318182
Ayamonte,1250.0,4.0,12.0,21104.0,18.953753,56.861259,2.0,0.333333
Alosno,154.0,4.0,11.0,3933.0,101.703534,279.684719,2.0,0.363636
Punta Umbría,672.0,6.0,14.0,15355.0,39.075220,91.175513,0.0,0.428571
Paterna del Campo,123.0,4.0,9.0,3457.0,115.707261,260.341336,0.0,0.444444
Palma del Condado (La),999.0,5.0,11.0,10801.0,46.292010,101.842422,1.0,0.454545
